# Pyrite Source Model

This notebook contains the code for modelling pyrite sources.

The code is contained in this notebook, below, and it provides for training and saving a model, given some input chemical data along with sources, or predicting sources, just given the chemical data. 

The code implements 2 types of model:
1. a linear, [Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) model
2. a non-linear, "tree" [Random Forest](https://en.wikipedia.org/wiki/Random_forest) model


When you train a model it will output to the screen the score of the model, the percentage of predictions it gets right. The Logistic Regression model scores ~88% (there's a random element to the scores, you won't get the same thing every time), the Random Forest is better, scoring ~96%. The Logistic Regression model has the benefit of being easier to interpret, as explained below.  

The linear model is the default. In both cases I've trained a model already, and they are in the **artifacts** folder, they are called ```model_linear.pkl``` and ```model_tree.pkl```. If an **artifacts** folder doesn't exist the code will make one.

When you train a model this also creates a *feature importances* file in the **artifacts** folder. Again, I've already trained one model of each so these files are in the folder.

---

## Feature Importances

These files contain the information on how the model uses the input to predict the Source.

**feature_importances_linear.csv**: This file has a row for each Source, and a column for each of the features it uses to make the predictions. You can easily interpret these values, for a given row the value for a given chemical tells you how this chemical changes the prediction. 

For example, for Source A and Fe we have the value, -2.643681213392997. This means that increasing the amount of Fe decreases the likelihood that a given sample comes from Source A. For Source A and S we have, 3.953436861312359. This means that increasing the amount of S increases the probability that a sample comes from Source A, and it has a larger effect than Fe (since the absolute value is greater).

**feature_importances_tree.csv**: This file has a row and a value for each chemical. These values are less easy to interpret, all we can say is that the larger the absolute value, the more important that chemical is in making the prediction, i.e. the stronger the relationship between this chemical and Source.

---

## Inputs and Outputs

To train a model the code expects input in a csv file (formatted like the one in the **train** folder). It should have a column called *Source*, and it will try to learn to predict this using the other columns. It requires the other columns to be called:

Fe, S, Co, Cu, Zn, As, Se, Mo, Ag, Sb, Pb, Stoichiometry

To make a prediction it expects all of the columns above, except **Source**, in a file formatted like the training file. It will output a file called **output.csv**, which is exactly the same as the input file but with an added column called **Prediction**.




# How to use

To train a model (which I've already done on the data you've provided, you won't need to redo this unless you're using new data), set ```mode``` (in the cell below) equal to 'train'. For predicting, set it to 'predict'.

Next decide what kind of model (as described above) you want to train/predict with. For Logistic Regresion set ```model_type``` equal to 'linear', for the Random Forest model set it to 'tree'. 

Finally add the location of the data that you want to train/predict with, set ```filename``` to this path. By path I mean the **full** fodler and filename of the data you want to use, something like: 'C:\Windows\User\Desktop\train.csv'. (I've set it to 'train/train.csv', this works on my machine but will **not** work on yours. 'train\train.csv' might, but the best bet is to go to the file and click on it, I think that shows the full path).

In [13]:
mode = 'predict'
model_type = "linear"
filename = r"C:\Users\dorna\Documents\Trace element paper\Modelling\pyrite\pyrite\train\train.csv"

Below is the code, you shouldn't need to edit any of this. To run the model just go to *Cell* in the menu bar, and select *Run all*. The results will appear at the bottom, along with the output information.

In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import ExtraTreeClassifier
from sklearn.model_selection import cross_val_score
import pickle
import warnings
import argparse
import os
warnings.filterwarnings('ignore')


feats = ["Fe", "S", "Co", "Cu", "Zn", "As", "Se", "Mo", "Ag", "Sb", "Pb", "Stoichiometry", "Sum"]
target = ["Source"]

def read(fname, cols):
    df = pd.read_csv(fname)
    assert all(c in df.columns for c in cols), \
        f"Not all columns found, all of {cols} needed for training"

    return df

def make_pipe(kind):
    assert kind is "tree" or kind is "linear", \
            f"--kind must be tree or linear, got {kind}"
    if kind is "linear":
        clf = ("clf", LogisticRegression())
    elif kind is "tree":
        clf = ("clf", ExtraTreeClassifier())

    pipe = Pipeline([("Scaler", StandardScaler()), clf])
    return pipe


def train(df, kind):
    pipe = make_pipe(kind)
    pipe.fit(df[feats], df[target])
    return pipe


def evaluate(df, model):
    scores = cross_val_score(model, df[feats], df[target],
            scoring="accuracy", cv=10)
    print(f"The average score on 10-fold CV was {100*scores.mean():.2f}%")

def make_dir():
    if 'artifacts' not in os.listdir():
        os.mkdir('artifacts')
    
def full_train(fname, kind):
    df = read(fname, target + feats)
    model = train(df, kind)
    pickle.dump(model, open(f"artifacts/model_{kind}.pkl", "wb"))
    evaluate(df, model)
    
    make_dir()
    with open(f"artifacts/feature_importances_{kind}.csv", "w") as f:
        if kind is "tree":
            imp = model.steps[1][1].feature_importances_
            for i, c in zip(imp, feats):
                f.write(f"{c}, {i}\n")

        else:
            imp = model.steps[1][1].coef_
            f.write(f"{','.join(c for c in target + feats)}\n")
            for name, row in zip(model.classes_, imp):
                f.write(f"{name}, {','.join([str(r) for r in row])}\n")


def predict(fname, kind):
    model = pickle.load(open(f"artifacts/model_{kind}.pkl", "rb"))
    df = read(fname, feats)
    df["Prediction"] = model.predict(df[feats])
    df[[f"Probability_{n}" for n in model.classes_]] = pd.DataFrame(model.predict_proba(df[feats]))

    df.to_csv("output.csv", index=False)
    print(f'Prediction complete, model predictions saved to {os.getcwd() + "/output.csv"}')                    
    

In [15]:
if mode is 'predict':
    predict(filename, model_type)
elif mode is 'train':
    full_train(filename, model_type)
else:
    print(f"mode must be 'train' or predict, it was set to {mode}")

Prediction complete, model predictions saved to C:\Users\dorna\Documents\Trace element paper\Modelling\pyrite\pyrite/output.csv
